In [24]:
# Install the necessary libraries
# !pip install xgboost
# !pip install seaborn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

from google.colab import drive
import gdown
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [25]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
file_id = '1hczD_xlMIzTDUOqHBGtuYdE4SwhabRtF'
destination = '/content/data.csv'  # Path where the file will be saved

In [27]:
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)
data=pd.read_csv(destination)

Downloading...
From: https://drive.google.com/uc?id=1hczD_xlMIzTDUOqHBGtuYdE4SwhabRtF
To: /content/data.csv
100%|██████████| 41.5k/41.5k [00:00<00:00, 36.5MB/s]


In [28]:
data.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [29]:
le = LabelEncoder()
data['sex'] = le.fit_transform(data['sex'])
data['smoker'] = le.fit_transform(data['smoker'])
data['region'] = le.fit_transform(data['region'])

In [30]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [31]:
data.corr()

,age,sex,bmi,children,smoker,region,charges
age,1.000000,-0.049141,0.116272,0.031256,-0.004548,0.001517,0.330647
sex,-0.049141,1.000000,0.009645,0.021033,0.080716,-0.007380,0.039561
bmi,0.116272,0.009645,1.000000,0.025760,-0.013054,0.146354,0.188470
children,0.031256,0.021033,0.025760,1.000000,-0.003532,0.000361,0.052630
smoker,-0.004548,0.080716,-0.013054,-0.003532,1.000000,-0.012993,0.784477
region,0.001517,-0.007380,0.146354,0.000361,-0.012993,1.000000,-0.027937
charges,0.330647,0.039561,0.188470,0.052630,0.784477,-0.027937,1.000000


In [32]:
data.isnull().sum()

,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


In [33]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [34]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)


In [35]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree=0.9, learning_rate=0.1,
                max_depth=5, alpha=10, n_estimators=100)
xg_reg.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [36]:
y_pred = xg_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [37]:
print(f"XGBoost R-squared: {r2}")

XGBoost R-squared: 0.8500580644071793


In [38]:
# Display actual vs predicted charges
result = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(result)

          Actual     Predicted
521   3994.17780   4579.207520
737   3484.33100   4056.396484
740   8604.48365  10512.642578
660   6435.62370  10353.123047
411  19594.80965  21612.363281
..           ...           ...
109  47055.53210  44424.960938
430  23082.95533   1340.719482
77    1532.46970   5307.778809
84   39836.51900  39225.906250
286   9432.92530  11809.366211

[250 rows x 2 columns]
